In [8]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import os
from datetime import datetime
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.font_manager as font_manager
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter
import quandl
from sodapy import Socrata
import seaborn as sns

#Clave API para bajar datos de QUANDL
quandl.ApiConfig.api_key = "AA2RGGesxh-s7q4qYFHz"

#Parámetros de fechas
start = datetime(2000, 1, 1)
end = datetime(2020, 12, 22)


In [2]:
#----------------------------------------------DATOS FRED------------------------------------------------#

#Datos de volatilidad
vix = web.DataReader('VIXCLS', 'fred', start, end)
gold_vol = web.DataReader('GVZCLS', 'fred', start, end)
oil_vol = web.DataReader('OVXCLS', 'fred', start, end)
em_vol = web.DataReader('VXEEMCLS', 'fred', start, end)
eurusd_vol = web.DataReader('EVZCLS', 'fred', start, end)

vix['Log_Vix'] = np.log(vix['VIXCLS'])

#Datos de spreads de crédito

AAA_UScorp_spread = web.DataReader('BAMLC0A1CAAA', 'fred', start, end)
AA_UScorp_spread = web.DataReader('BAMLC0A2CAA', 'fred', start, end)
BBB_UScorp_spread = web.DataReader('BAMLC0A4CBBB', 'fred', start, end)
US_HY_spread = web.DataReader('BAMLH0A0HYM2', 'fred', start, end)
LATAM_corp_spread = web.DataReader('BAMLEMRLCRPILAOAS', 'fred', start, end)

BBB_UScorp_yield = web.DataReader('BAMLC0A4CBBBEY', 'fred', start, end)


#Datos de tasas de interés
#Treasuries
US_2Y = web.DataReader('DGS2', 'fred', start, end)
US_5Y = web.DataReader('DGS5', 'fred', start, end)
US_10Y = web.DataReader('DGS10', 'fred', start, end)
US_30Y = web.DataReader('DGS30', 'fred', start, end)
US_slope_10Y2Y = web.DataReader('T10Y2Y', 'fred', start, end)
#Break Even Inflation
US_BEI_5Y = web.DataReader('T5YIE', 'fred', start, end)
US_BEI_10Y = web.DataReader('T10YIE', 'fred', start, end)
US_BEI_5Y5Y = web.DataReader('T5YIFR', 'fred', start, end)
#Real rates
US_TIP_10Y= web.DataReader('DFII10', 'fred', start, end)
#Instantaneous forward rate
US_FWD_1D1Y= web.DataReader('THREEFF1', 'fred', start, end)

#Datos de tasas de cambio
#Índices del dólar
USD_INDEX_EM =  web.DataReader('DTWEXEMEGS', 'fred', start, end)
USD_INDEX_DEV =  web.DataReader('DTWEXAFEGS', 'fred', start, end)

#Monedas fuertes
EURUSD =  web.DataReader('DEXUSEU', 'fred', start, end)
GBPUSD =  web.DataReader('DEXUSUK', 'fred', start, end)
USDCNH =  web.DataReader('DEXCHUS', 'fred', start, end)
USDJPY =  web.DataReader('DEXJPUS', 'fred', start, end)
USDCAD =  web.DataReader('DEXCAUS', 'fred', start, end)

#Monedas emergentes
USDKRW =  web.DataReader('DEXKOUS', 'fred', start, end)
USDMXN =  web.DataReader('DEXMXUS', 'fred', start, end)
USDBRL =  web.DataReader('DEXBZUS', 'fred', start, end)
USDINR =  web.DataReader('DEXINUS', 'fred', start, end)
USDZAR =  web.DataReader('DEXSFUS', 'fred', start, end)




#Datos Macro US
US_GDP =  web.DataReader('GDP', 'fred', start, end)
US_GDPNOW=  web.DataReader('GDPNOW', 'fred', start, end)  #Atlanta FED
US_GDPNOWCAST =  web.DataReader('STLENI', 'fred', start, end)  #St Louis FED

US_FED_TOTAL_ASSETS=  web.DataReader('WALCL', 'fred', start, end)  
US_FED_LIQUIDITY_SWAPS =  web.DataReader('SWPT', 'fred', start, end)  


US_PUBLIC_DEBT_GDP_PERC =  web.DataReader('GFDEGDQ188S', 'fred', start, end)
US_HOUSEHOLD_DEBT_GDP_PERC =  web.DataReader('HDTGPDUSQ163N', 'fred', start, end)
US_CORP_PROFITS =  web.DataReader('CP', 'fred', start, end)

#Datos Macro EZ
US_ECB_TOTAL_ASSETS=  web.DataReader('ECBASSETSW', 'fred', start, end)  


#----------------------------------------------DATOS QUANDL------------------------------------------------#

#Futuros materias primas
COPPER = quandl.get("CHRIS/CME_HG1",start_date=start, end_date=end)
GOLD = quandl.get("CHRIS/CME_GC1",start_date=start, end_date=end)
OIL = quandl.get("CHRIS/CME_CL1",start_date=start, end_date=end)
COPPER['RATIO'] = COPPER['Last']*100/GOLD['Last']

#Futuros monedas
EURUSD = quandl.get("CHRIS/CME_EC1",start_date=start, end_date=end)

#Tasas de interés
ACM_TERM_PREMIA = quandl.get("FRBNY/ACM_D",start_date=start, end_date=end)


#Macro US
ISM_EXPORTS = quandl.get("ISM/NONMAN_EXPORTS",start_date=start, end_date=end)
ISM_COMPOSITE = quandl.get("ISM/MAN_PMI",start_date=start, end_date=end)


#Tasa de cambio real - datos BIS
USD_REER = quandl.get("BIS/EM_MRNUS",start_date=start, end_date=end)
COP_REER = quandl.get("BIS/EM_MRBCO",start_date=start, end_date=end)
CLP_REER = quandl.get("BIS/EM_MRBCL",start_date=start, end_date=end)
MXN_REER = quandl.get("BIS/EM_MRBMX",start_date=start, end_date=end)
BRL_REER = quandl.get("BIS/EM_MRBBR",start_date=start, end_date=end)
ARS_REER = quandl.get("BIS/EM_MRBAR",start_date=start, end_date=end)

In [9]:
client = Socrata("www.datos.gov.co",
                 "Oy0pJTAMwxGuC0c5SBC9nuuAU",
                 username="43xcm1yx3xh1eb5lv8e374g1s",
                 password="1zt8gsxe1mfh3h6fyh6281n1qitip816l2d6wnc1zvvs2dyidz")

USDCOP_TRM = client.get("mcec-87by", limit=10000)
USDCOP_TRM_df = pd.DataFrame.from_records(USDCOP_TRM)
USDCOP_TRM_df.sort_values('vigenciadesde')

,valor,unidad,vigenciadesde,vigenciahasta
6992,643.42,COP,1991-12-02T00:00:00.000,1991-12-02T00:00:00.000
6991,639.22,COP,1991-12-03T00:00:00.000,1991-12-03T00:00:00.000
6990,635.70,COP,1991-12-04T00:00:00.000,1991-12-04T00:00:00.000
6989,631.51,COP,1991-12-05T00:00:00.000,1991-12-05T00:00:00.000
6988,627.16,COP,1991-12-06T00:00:00.000,1991-12-06T00:00:00.000
...,...,...,...,...
4,3416.21,COP,2020-12-17T00:00:00.000,2020-12-17T00:00:00.000
3,3410.82,COP,2020-12-18T00:00:00.000,2020-12-18T00:00:00.000
2,3420.26,COP,2020-12-19T00:00:00.000,2020-12-21T00:00:00.000
1,3442.41,COP,2020-12-22T00:00:00.000,2020-12-22T00:00:00.000


In [10]:
client = Socrata("www.datos.gov.co",
                 "Oy0pJTAMwxGuC0c5SBC9nuuAU",
                 username="43xcm1yx3xh1eb5lv8e374g1s",
                 password="1zt8gsxe1mfh3h6fyh6281n1qitip816l2d6wnc1zvvs2dyidz")

VU_FONDOS_PENSIONES_OBL = client.get("uawh-cjvi", limit=200000)
VU_FONDOS_PENSIONES_OBL_df = pd.DataFrame.from_records(VU_FONDOS_PENSIONES_OBL)
VU_FONDOS_PENSIONES_OBL_df.sort_values('fecha')

,fecha,codigo_entidad,nombre_entidad,codigo_patrimonio,nombre_fondo,valor_unidad
0,1999-03-01T00:00:00.000,1,Pensiones Y Cesantias Colpatria,1,Fondo de Cesantias Largo Plazo,5284.010000
15,1999-03-01T00:00:00.000,10,Colfondos S.A. Pensiones Y Cesantias,1000,Fondo de Pensiones Moderado,3827.000000
14,1999-03-01T00:00:00.000,10,Colfondos S.A. Pensiones Y Cesantias,1,Fondo de Cesantias Largo Plazo,5331.870000
13,1999-03-01T00:00:00.000,9,Skandia Pensiones Y Cesantías S.A.,1000,Fondo de Pensiones Moderado,3154.430000
12,1999-03-01T00:00:00.000,9,Skandia Pensiones Y Cesantías S.A.,1,Fondo de Cesantias Largo Plazo,5781.150000
...,...,...,...,...,...,...
156651,2020-12-19T00:00:00.000,2,Proteccion,2,Fondo de Cesantias Corto Plazo,29701.980000
156650,2020-12-19T00:00:00.000,2,Proteccion,1,Fondo de Cesantias Largo Plazo,31725.380000
156673,2020-12-19T00:00:00.000,10,Colfondos S.A. Pensiones Y Cesantias,6000,Fondo de Pensiones Mayor Riesgo,56894.540000
156661,2020-12-19T00:00:00.000,3,Porvenir,7000,Fondo de Pensiones Retiro Programado,58413.010000


In [7]:
#CBONDS

# # library for making requests to a web service
# import requests
# # library for working with data
# import pandas as pd
# # connect a method that converts a JSON string to a table
# from pandas.io.json import json_normalize
# # library for delay between requests
# import time


# url = 'https://ws.cbonds.info/services/json?login=Test&password=Test'
# # url = 'https://ws.cbonds.info/services/json?login=Test&password=Test&lang=rus' - in Russian
# # request a JSON schema describing all available web service operations
# response = requests.get(url).json()
# json_scheme = json_normalize(response)
# # get a description of the get emissions operation
# # emissions_scheme = json_normalize(json_scheme['service.get_emissions.arguments.filters'][0])
# index_scheme = json_normalize(json_scheme['service.get_index_value.arguments.filters'][0])
# # description output
# # print(emissions_scheme)
# # print(index_scheme)

# index_scheme

,field,type,available_operators,title,sortable,searchable
0,id,int,"[eq, ne, nu, nn, lt, le, gt, ge, in, ni]",Unique identifier,0,0
1,date,date,"[eq, ne, nu, nn, lt, le, gt, ge, in, ni]",Index value date,1,1
2,index_group_id,int,"[eq, ne, nu, nn, lt, le, gt, ge, in, ni]",Index group (id),0,0
3,index_group2_id,int,"[eq, ne, nu, nn, lt, le, gt, ge, in, ni]",Index group2 (id),0,0
4,index_group_name,string,"[eq, ne, nu, nn, bw, bn, ew, en, cn, nc, in, ni]",Index group (rus),0,0
5,time,date,"[eq, ne, nu, nn, lt, le, gt, ge, in, ni]",Time,0,0
6,type_id,int,"[eq, ne, nu, nn, lt, le, gt, ge, in, ni]",Index type (id),1,1
7,type_name_eng,string,"[eq, ne, nu, nn, bw, bn, ew, en, cn, nc, in, ni]",Index type (eng),0,0
8,type_name_ita,string,"[eq, ne, nu, nn, bw, bn, ew, en, cn, nc, in, ni]",Index type (ita),0,0
9,type_name_pol,string,"[eq, ne, nu, nn, bw, bn, ew, en, cn, nc, in, ni]",Index type (pol),0,0


In [ ]:
# symbol = 'FB.US'
# quandl = web.DataReader(symbol, 'quandl', '2015-01-01')

#EXAMPLES
# oil = quandl.get('EIA/PET_RWTC_D',start_date="2001-12-31", end_date="2005-12-31").squeeze()
# oil = quandl.get('EIA/PET_RWTC_D',start_date="2000-12-31", end_date="2020-12-18")

# To request specific columns:

# mydata = quandl.get(["NSE/OIL.1", "WIKI/AAPL.4"])
# To request the last n rows:

# mydata = quandl.get("WIKI/AAPL", rows=5)

# To change the sampling frequency:

# mydata = quandl.get("EIA/PET_RWTC_D", collapse="monthly")

# FB = quandl.get('WIKI/FB',start_date="2000-12-31", end_date="2020-12-18")

# data = quandl.get_table('MER/F1', compnumber="39102", paginate=True)
# quandl.get_table('MER/F1', paginate=True)